In [4]:
import numpy as np
import pylab as pl 
import matplotlib.pyplot as plt
%matplotlib inline

# Neural Networks

In [382]:
class Neuralnets(object):
    
    def __init__(self,classes):
        self.classes = classes
        
    
    # Initialize the weights as a numpy array (All weights will be matrices)
    # Initialize the bias terms as a numpy array as well (Bias terms will be vectors)
    
    def buildNeuralNetworks(self,X,y,weightStorage,biasStorage,hL,nodes,iteration,eta):
        # SGD
        
        newWeight = weightStorage
        newBias = biasStorage
        WeightList = []
        biasList = []
        for i in xrange(0,iteration):
            #np.random.shuffle(X)
            samples,features = X.shape
            for j in xrange(0,samples):
                #print j
                #print '---'
                x = X[j,]
            
                # Feed Forward
                # Store W,a,f'(z)
                #weightStorage = np.empty()
                valueStorage = []
                activeDerivStorage = []
                #a = x
                valueStorage.append(x)
                #print x
                for k in range(hL+1):
                    
                    z =np.dot(np.transpose(newWeight[k]),valueStorage[k]) +newBias[k]
                    #print '--z-value--'
                    #print z
                    if k != hL:
                        #print z
                        f = np.vectorize(self.ReLU)
                        a = f(z)
                        
   
                        
                        valueStorage.append(a)
                        #print valueStorage.shape
                        fPrime = np.vectorize(self.ReLUDeriv)
                        activeDerivStorage.append(fPrime(z))
                    else:
                        #f = np.vectorize(self.softmax)
                        a = self.softmax(z)
                        
                        valueStorage.append(a)
                        #fPrime = np.vectorize(self.softmaxDeriv)
                        activeDerivStorage.append(self.softmaxDeriv(z))

                outputError =  valueStorage[hL+1] - y[j]
                
                deltaStorage = [None]*(hL+1)
                deltaStorage[hL] = outputError
                for l in reversed(range(hL)):

                    delta = np.dot(np.dot(np.diag(activeDerivStorage[l]),newWeight[l+1]),deltaStorage[l+1])
                    deltaStorage[l] = np.array(delta)
                    #print deltaStorage
                 
                updatedWeights = []
                updatedBias = []
                for k in range(0,hL+1):

                    weightGradient = np.outer(valueStorage[k],deltaStorage[k])
                    #print weightGradient
                    biasGradient = deltaStorage[k]
                    newWeight[k] = newWeight[k] - (eta*weightGradient)
                    updatedWeights.append(newWeight[k])
                    #print '--weight Matrices--'
                    #print newWeight[k]
                    newBias[k] = newBias[k] - (eta*biasGradient)
                    updatedBias.append(newBias[k])

                WeightList.append(updatedWeights) 
                biasList.append(updatedBias)  
                #print newWeight
        return (WeightList[-1],biasList[-1])
                    
    
    
    def predictNN(self,X,weight,bias):

        predictions = []
        for j in range(X.shape[0]):
            a = [np.transpose(np.array([X[j,:]]))]
            #print a
            #y = np.array([Y[j]])
            z = []
            
            for k in range(len(weight)):
                
                z.append(np.array(np.dot(np.transpose(weight[k]), a[k])+np.transpose(np.array([bias[k]]))))
                if k < (len(weight)-1):
                    #print z[k]
                    f = np.vectorize(self.ReLU)
                    #print f(z[k])
                    a.append(np.array(f(z[k])))
                    #print z[k]
                else:
                    #print z[k]
                    a.append(np.array(self.softmax(z[k])))

            max_prob = np.max(a[len(weight)])
            #print max_prob
            #print list(a[len(weight)])
            max_index = list(a[len(weight)]).index(max_prob)
            prediction = np.zeros(len(a[len(weight)]))
            prediction[max_index] = 1
            predictions.append(prediction)
        #classification_accuracy = sum([all(predictions[i]==array(Y)[i]) for i in range(len(Y))])/len(Y)
        return(predictions)
        
    def ClassificationError(self,pred,Y):
        """ Provides the misclassification error given the true value and the prediction"""
        
        cError = 1 - sum([all(pred[i]==Y[i]) for i in range(len(Y))])/np.float(len(Y))
        return(cError)
                
    def ParameterInitialization(self,X,y,nodes):
        # Control the number of nodes in the weight initialization function
        # nodes variable should be a list of number of nodes in each hidden layer
        weightList = []
        biasList = []
        allNodes = [X.shape[1]]+nodes+[y.shape[1]]
        
        for i in range(1,len(allNodes)):
            weight = np.empty([allNodes[i-1],allNodes[i]])
            # Gaussian random weights with variance 1/sqrt(m)
            weight = np.reshape(np.random.normal(0, 1/np.sqrt(allNodes[i-1]),weight.size), (allNodes[i-1],allNodes[i]))
            weightList.append(weight)
            
            bias = np.random.normal(0,1/np.sqrt((allNodes[i])),allNodes[i])
            biasList.append(bias)
        return(weightList,biasList)
            
    
    
    def ReLU(self,z):
        return np.amax([0,z])
    
    def softmax(self,Z):
            #denom = sum(np.exp(Z))
        #return(np.exp(Z)/denom)
        e_x = np.exp(Z - np.max(Z))
        return e_x / e_x.sum(axis=0)

    
    def ReLUDeriv(self,z):
        if z>0:
            Derive = 1
        else:
            # subgradient at zero was set to zero
            Derive = 0
        return(Derive)
    
    def softmaxDeriv(self,Z):
        n = len(Z)
        denom = sum(np.exp(Z))
        
        Jacobian = np.empty([n,n])
        for i in range(n):
            for j in range(i+1):
                if i == j:
                    Jacobian[i,j] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
                    Jacobian[j,i] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
                else:
                    Jacobian[i,j] = (np.exp(-(Z[i]+Z[j]))/denom)**2
                    Jacobian[j,i] = (np.exp(-(Z[i]+Z[j]))/denom)**2
        return (Jacobian)
                
    def crossEntropyLoss(self, y,guess):
        return(-(y/guess))
        
    

### Using the given data_3class.csv

In [7]:
train = pl.loadtxt('../DataFiles/HW3/data_3class.csv')


X = train[:, 0:2]
Y = train[:, 2:3]
Y
len(Y)
Y_encode = np.int_(np.reshape(Y,(800,)))
b = np.zeros((len(Y_encode), 3))
b[np.arange(len(Y_encode)), Y_encode] = 1



In [384]:
X_sub = X[0:49,]
b_sub = b[0:49]

In [393]:
NNexercise_data3 = Neuralnets(3)
nodes = [3]
NNexercise_data3.softmax(np.array([[2.3],[3.4]]))
Weights,bias = NNexercise_data3.ParameterInitialization(X_sub,b_sub,nodes)
#print Weights,bias
#Weights2,bias2 = NNexercise_data3.buildNeuralNetworks(X_sub,b_sub,Weights,bias,1,nodes,1,1)
W,B = NNexercise_data3.buildNeuralNetworks(X_sub,b_sub,Weights,bias,1,nodes,500,0.0001)

In [401]:
pred = NNexercise_data3.predictNN(X_sub,W,B)
#print pred

In [402]:
NNexercise_data3.ClassificationError(pred,b_sub)

0.4897959183673469

In [183]:
Weights,bias = NNexercise_data3.ParameterInitialization(X,b,nodes)
Weights2,bias2 = NNexercise_data3.ParameterInitialization(X,b,nodes)

In [184]:
pred = NNexercise_data3.predictNN(X,Weights,bias)

In [185]:
pred2 = NNexercise.predictNN(X,Weights2,bias2)

NameError: name 'NNexercise' is not defined

In [279]:
NNexercise.ClassificationError(pred2,b)

0.66875

In [236]:


NNexercise_data3 = Neuralnets(3)
nodes = [3]
#Params = NNexercise_data3.ParameterInitialization(X,b,nodes)
#print Params[0]
Weight,bias = NNexercise_data3.buildNeuralNetworks(X,b,1,nodes,100,0.001)
#print bias

TypeError: buildNeuralNetworks() takes exactly 9 arguments (7 given)

In [209]:
Weight

[array([[-0.59000996, -1.17710391,  1.18395388],
        [-1.2303208 , -1.19403115, -0.70056632]]),
 array([[ 0.01547935, -0.04329687,  0.06930713],
        [ 0.03615068,  0.21948215,  0.01819244],
        [ 0.02567221,  0.18030655,  0.19062912]])]

In [9]:
#NNexercise.predictNN(X,Weight,bias)
pred = NNexercise.predictNN(X_sub,Weight,bias)
#pred

In [235]:
NNexercise.ClassificationError(pred,b_sub)

0.4

In [10]:
#b_sub\
pred = NNexercise.predictNN(X_sub,Weight,bias)


In [213]:
NNexercise.ClassificationError(pred,b_sub)

0.8

In [11]:
# Vectorized functions

def ReLU(z):
    return(np.amax([0,z]))

vReLU = np.vectorize(ReLU)

def softmax(z):
    z = np.array(z)-np.max(z)
    return(np.exp(z)/sum(np.exp(z)))

## Trial Runs

In [52]:
#One small  hidden layer - 3 
#One large hidden layer - 50 
#Two small hidden layer - [3,4] 
#Two large hidden layer - [50,60] 

In [53]:
# 2D Datasets

In [54]:
# Dataset 1

data = pl.loadtxt('../DataFiles/HW2/data1_train.csv')
len(data[:,2])
Y = data[:,2]
Y = Y *0.5+0.5
Y = np.int_(np.reshape(Y,(len(Y,))))
b = np.zeros((len(Y), 2))
b[np.arange(len(Y)), Y] = 1
b
Y1_Train = b
X1_Train = data[:,0:2]




data1_V = pl.loadtxt('../DataFiles/HW2/data1_validate.csv')
Y1_V = data1_V[:,2]
Y1_V = Y1_V *0.5+0.5
Y1_V = np.int_(np.reshape(Y1_V,(len(Y1_V,))))
b = np.zeros((len(Y1_V), 2))
b[np.arange(len(Y1_V)), Y1_V] = 1
b
Y1_V = b
X1_V = data1_V[:,0:2]



data1_Test = pl.loadtxt('../DataFiles/HW2/data1_test.csv')
Y1_Test = data1_Test[:,2]
Y1_Test = Y1_Test *0.5+0.5
Y1_Test = np.int_(np.reshape(Y1_Test,(len(Y1_Test,))))
b = np.zeros((len(Y1_Test), 2))
b[np.arange(len(Y1_Test)), Y1_Test] = 1
b
Y1_Test = b
X1_Test = data1_Test[:,0:2]



In [55]:
# Dataset 2

data = pl.loadtxt('../DataFiles/HW2/data2_train.csv')
len(data[:,2])
Y2_Train = data[:,2]
Y2_Train = Y2_Train *0.5+0.5
Y2_Train = np.int_(np.reshape(Y2_Train,(len(Y2_Train,))))
b = np.zeros((len(Y2_Train), 2))
b[np.arange(len(Y2_Train)), Y2_Train] = 1
b
Y2_Train = b
X2_Train = data[:,0:2]




data_V = pl.loadtxt('../DataFiles/HW2/data2_validate.csv')
Y2_V = data_V[:,2]
Y2_V = Y2_V *0.5+0.5
Y2_V = np.int_(np.reshape(Y2_V,(len(Y2_V,))))
b = np.zeros((len(Y2_V), 2))
b[np.arange(len(Y2_V)), Y2_V] = 1
b
Y2_V = b
X2_V = data_V[:,0:2]



data_Test = pl.loadtxt('../DataFiles/HW2/data1_test.csv')
Y2_Test = data_Test[:,2]
Y2_Test = Y2_Test *0.5+0.5
Y2_Test = np.int_(np.reshape(Y2_Test,(len(Y2_Test,))))
b = np.zeros((len(Y2_Test), 2))
b[np.arange(len(Y2_Test)), Y2_Test] = 1
b
Y2_Test = b
X2_Test = data_Test[:,0:2]



In [56]:
# Dataset 3

data = pl.loadtxt('../DataFiles/HW2/data3_train.csv')
len(data[:,2])
Y3_Train = data[:,2]
Y3_Train = Y3_Train *0.5+0.5
Y3_Train = np.int_(np.reshape(Y3_Train,(len(Y3_Train,))))
b = np.zeros((len(Y3_Train), 2))
b[np.arange(len(Y3_Train)), Y3_Train] = 1
b
Y3_Train = b
X3_Train = data[:,0:2]




data_V = pl.loadtxt('../DataFiles/HW2/data3_validate.csv')
Y3_V = data_V[:,2]
Y3_V = Y3_V *0.5+0.5
Y3_V = np.int_(np.reshape(Y3_V,(len(Y3_V,))))
b = np.zeros((len(Y3_V), 2))
b[np.arange(len(Y3_V)), Y3_V] = 1
b
Y3_V = b
X3_V = data_V[:,0:2]



data_Test = pl.loadtxt('../DataFiles/HW2/data3_test.csv')
Y3_Test = data_Test[:,2]
Y3_Test = Y3_Test *0.5+0.5
Y3_Test = np.int_(np.reshape(Y3_Test,(len(Y3_Test,))))
b = np.zeros((len(Y3_Test), 2))
b[np.arange(len(Y3_Test)), Y3_Test] = 1
b
Y3_Test = b
X3_Test = data_Test[:,0:2]



In [60]:
# Dataset 4

data = pl.loadtxt('../DataFiles/HW2/data4_train.csv')
len(data[:,2])
Y4_Train = data[:,2]
Y4_Train = Y4_Train *0.5+0.5
Y4_Train = np.int_(np.reshape(Y4_Train,(len(Y4_Train,))))
b = np.zeros((len(Y4_Train), 2))
b[np.arange(len(Y4_Train)), Y4_Train] = 1
b
Y4_Train = b
X4_Train = data[:,0:2]




data_V = pl.loadtxt('../DataFiles/HW2/data4_validate.csv')
Y4_V = data_V[:,2]
Y4_V = Y4_V *0.5+0.5
Y4_V = np.int_(np.reshape(Y4_V,(len(Y4_V,))))
b = np.zeros((len(Y4_V), 2))
b[np.arange(len(Y4_V)), Y4_V] = 1
b
Y4_V = b
X4_V = data_V[:,0:2]



data_Test = pl.loadtxt('../DataFiles/HW2/data4_test.csv')
Y4_Test = data_Test[:,2]
Y4_Test = Y4_Test *0.5+0.5
Y4_Test = np.int_(np.reshape(Y4_Test,(len(Y4_Test,))))
b = np.zeros((len(Y4_Test), 2))
b[np.arange(len(Y4_Test)), Y4_Test] = 1
b
Y4_Test = b
X4_Test = data_Test[:,0:2]



In [67]:
# One small hidden layer 
NNexercise_data3 = Neuralnets(3)
nodes = [3]
Weight,bias = NNexercise_data3.buildNeuralNetworks(X1_Train,Y1_Train,1,nodes,500,0.0001)
pred = NNexercise.predictNN(X1_Train,Weight,bias)
NNexercise.ClassificationError(pred,Y1_Train)

0.5025

In [69]:
pred = NNexercise.predictNN(X1_Test,Weight,bias)
NNexercise.ClassificationError(pred,Y1_Test)

0.48

In [70]:
# One large hidden layer 
NNexercise_data3 = Neuralnets(3)
nodes = [50]
Weight,bias = NNexercise_data3.buildNeuralNetworks(X1_Train,Y1_Train,1,nodes,100,0.0001)
pred = NNexercise.predictNN(X1_Train,Weight,bias)
NNexercise.ClassificationError(pred,Y1_Train)

0.5125

In [46]:
# MNIST dataset
# 10 class digit classification
# Atleast 200 training samples, validation and testing to be 150 samples

In [84]:
# Generating Train, Test and Validation set for this question

MNISTData_0 = pl.loadtxt('../DataFiles/HW2/mnist_digit_0.csv')
MNISTData_0_Train =MNISTData_0[:200,]
MNISTData_0_Valid =MNISTData_0[201:351,]
MNISTData_0_Test =MNISTData_0[351:501,]

MNISTData_1 = pl.loadtxt('../DataFiles/HW2/mnist_digit_1.csv')
MNISTData_1_Train =MNISTData_1[:200,]
MNISTData_1_Valid =MNISTData_1[201:351,]
MNISTData_1_Test =MNISTData_1[351:501,]

MNISTData_2 = pl.loadtxt('../DataFiles/HW2/mnist_digit_2.csv')
MNISTData_2_Train =MNISTData_2[:200,]
MNISTData_2_Valid =MNISTData_2[201:351,]
MNISTData_2_Test =MNISTData_2[351:501,]

MNISTData_3 = pl.loadtxt('../DataFiles/HW2/mnist_digit_3.csv')
MNISTData_3_Train =MNISTData_3[:200,]
MNISTData_3_Valid =MNISTData_3[201:351,]
MNISTData_3_Test =MNISTData_3[351:501,]

MNISTData_4 = pl.loadtxt('../DataFiles/HW2/mnist_digit_4.csv')
MNISTData_4_Train =MNISTData_4[:200,]
MNISTData_4_Valid =MNISTData_4[201:351,]
MNISTData_4_Test =MNISTData_4[351:501,]

MNISTData_5 = pl.loadtxt('../DataFiles/HW2/mnist_digit_5.csv')
MNISTData_5_Train =MNISTData_5[:200,]
MNISTData_5_Valid =MNISTData_5[201:351,]
MNISTData_5_Test =MNISTData_5[351:501,]

MNISTData_6 = pl.loadtxt('../DataFiles/HW2/mnist_digit_6.csv')
MNISTData_6_Train =MNISTData_6[:200,]
MNISTData_6_Valid =MNISTData_6[201:351,]
MNISTData_6_Test =MNISTData_6[351:501,]

MNISTData_7 = pl.loadtxt('../DataFiles/HW2/mnist_digit_7.csv')
MNISTData_7_Train =MNISTData_7[:200,]
MNISTData_7_Valid =MNISTData_7[201:351,]
MNISTData_7_Test =MNISTData_7[351:501,]

MNISTData_8 = pl.loadtxt('../DataFiles/HW2/mnist_digit_8.csv')
MNISTData_8_Train =MNISTData_8[:200,]
MNISTData_8_Valid =MNISTData_8[201:351,]
MNISTData_8_Test =MNISTData_8[351:501,]

MNISTData_9 = pl.loadtxt('../DataFiles/HW2/mnist_digit_9.csv')
MNISTData_9_Train =MNISTData_9[:200,]
MNISTData_9_Valid =MNISTData_9[201:351,]
MNISTData_9_Test =MNISTData_9[351:501,]

In [85]:
MNISTData_0_Train_Augmented = np.c_[np.array([0]*MNISTData_0_Train.shape[0]), MNISTData_0_Train]
MNISTData_0_Valid_Augmented = np.c_[np.array([0]*MNISTData_0_Valid.shape[0]), MNISTData_0_Valid]
MNISTData_0_Test_Augmented = np.c_[np.array([0]*MNISTData_0_Test.shape[0]), MNISTData_0_Test]

MNISTData_1_Train_Augmented = np.c_[np.array([1]*MNISTData_1_Train.shape[0]), MNISTData_1_Train]
MNISTData_1_Valid_Augmented = np.c_[np.array([1]*MNISTData_1_Valid.shape[0]), MNISTData_1_Valid]
MNISTData_1_Test_Augmented = np.c_[np.array([1]*MNISTData_1_Test.shape[0]), MNISTData_1_Test]

MNISTData_2_Train_Augmented = np.c_[np.array([2]*MNISTData_2_Train.shape[0]), MNISTData_2_Train]
MNISTData_2_Valid_Augmented = np.c_[np.array([2]*MNISTData_2_Valid.shape[0]), MNISTData_2_Valid]
MNISTData_2_Test_Augmented = np.c_[np.array([2]*MNISTData_2_Test.shape[0]), MNISTData_2_Test]

MNISTData_3_Train_Augmented = np.c_[np.array([3]*MNISTData_3_Train.shape[0]), MNISTData_3_Train]
MNISTData_3_Valid_Augmented = np.c_[np.array([3]*MNISTData_3_Valid.shape[0]), MNISTData_3_Valid]
MNISTData_3_Test_Augmented = np.c_[np.array([3]*MNISTData_3_Test.shape[0]), MNISTData_3_Test]

MNISTData_4_Train_Augmented = np.c_[np.array([4]*MNISTData_4_Train.shape[0]), MNISTData_4_Train]
MNISTData_4_Valid_Augmented = np.c_[np.array([4]*MNISTData_4_Valid.shape[0]), MNISTData_4_Valid]
MNISTData_4_Test_Augmented = np.c_[np.array([4]*MNISTData_4_Test.shape[0]), MNISTData_4_Test]

MNISTData_5_Train_Augmented = np.c_[np.array([5]*MNISTData_5_Train.shape[0]), MNISTData_5_Train]
MNISTData_5_Valid_Augmented = np.c_[np.array([5]*MNISTData_5_Valid.shape[0]), MNISTData_5_Valid]
MNISTData_5_Test_Augmented = np.c_[np.array([5]*MNISTData_5_Test.shape[0]), MNISTData_5_Test]

MNISTData_6_Train_Augmented = np.c_[np.array([6]*MNISTData_6_Train.shape[0]), MNISTData_6_Train]
MNISTData_6_Valid_Augmented = np.c_[np.array([6]*MNISTData_6_Valid.shape[0]), MNISTData_6_Valid]
MNISTData_6_Test_Augmented = np.c_[np.array([6]*MNISTData_6_Test.shape[0]), MNISTData_6_Test]


MNISTData_7_Train_Augmented = np.c_[np.array([7]*MNISTData_7_Train.shape[0]), MNISTData_7_Train]
MNISTData_7_Valid_Augmented = np.c_[np.array([7]*MNISTData_7_Valid.shape[0]), MNISTData_7_Valid]
MNISTData_7_Test_Augmented = np.c_[np.array([7]*MNISTData_7_Test.shape[0]), MNISTData_7_Test]

MNISTData_8_Train_Augmented = np.c_[np.array([8]*MNISTData_8_Train.shape[0]), MNISTData_8_Train]
MNISTData_8_Valid_Augmented = np.c_[np.array([8]*MNISTData_8_Valid.shape[0]), MNISTData_8_Valid]
MNISTData_8_Test_Augmented = np.c_[np.array([8]*MNISTData_8_Test.shape[0]), MNISTData_8_Test]

MNISTData_9_Train_Augmented = np.c_[np.array([9]*MNISTData_9_Train.shape[0]), MNISTData_9_Train]
MNISTData_9_Valid_Augmented = np.c_[np.array([9]*MNISTData_9_Valid.shape[0]), MNISTData_9_Valid]
MNISTData_9_Test_Augmented = np.c_[np.array([9]*MNISTData_9_Test.shape[0]), MNISTData_9_Test]

In [86]:
Set_train = np.vstack([MNISTData_1_Train_Augmented,MNISTData_2_Train_Augmented,MNISTData_3_Train_Augmented,MNISTData_4_Train_Augmented,
                      MNISTData_5_Train_Augmented,MNISTData_6_Train_Augmented,MNISTData_7_Train_Augmented,MNISTData_8_Train_Augmented,
                      MNISTData_9_Train_Augmented,MNISTData_0_Train_Augmented])
                      
                      
np.random.shuffle(Set_train)
Set_train_X = Set_train[:,1:]
Set_train_X_Norm = (2*Set_train_X/255)-1
Set_train_Y = Set_train[:,0]

Set_train_Y = np.int_(np.reshape(Set_train_Y,(len(Set_train_Y,))))
b = np.zeros((len(Set_train_Y), 10))
b[np.arange(len(Set_train_Y)), Set_train_Y] = 1
Set_train_Y = b

In [93]:
Set_valid = np.vstack([MNISTData_1_Valid_Augmented,MNISTData_2_Valid_Augmented,MNISTData_3_Valid_Augmented,MNISTData_4_Valid_Augmented,
                      MNISTData_5_Valid_Augmented,MNISTData_6_Valid_Augmented,MNISTData_7_Valid_Augmented,MNISTData_8_Valid_Augmented,
                      MNISTData_9_Valid_Augmented,MNISTData_0_Valid_Augmented])
                      
                      
np.random.shuffle(Set_valid)
Set_valid_X = Set_valid[:,1:]
Set_valid_X_Norm = (2*Set_valid_X/255)-1
Set_valid_Y = Set_valid[:,0]

Set_valid_Y = np.int_(np.reshape(Set_valid_Y,(len(Set_valid_Y,))))
b = np.zeros((len(Set_valid_Y), 10))
b[np.arange(len(Set_valid_Y)), Set_valid_Y] = 1
Set_valid_Y = b

In [95]:
Set_test = np.vstack([MNISTData_1_Test_Augmented,MNISTData_2_Test_Augmented,MNISTData_3_Test_Augmented,MNISTData_4_Test_Augmented,
                      MNISTData_5_Test_Augmented,MNISTData_6_Test_Augmented,MNISTData_7_Test_Augmented,MNISTData_8_Test_Augmented,
                      MNISTData_9_Test_Augmented,MNISTData_0_Test_Augmented])
                      
                      
np.random.shuffle(Set_test)
Set_test_X = Set_valid[:,1:]
Set_test_X_Norm = (2*Set_test_X/255)-1
Set_test_Y = Set_test[:,0]

Set_test_Y = np.int_(np.reshape(Set_test_Y,(len(Set_test_Y,))))
b = np.zeros((len(Set_test_Y), 10))
b[np.arange(len(Set_test_Y)), Set_test_Y] = 1
Set_test_Y = b

# Convolutional Neural Networks

In [2]:
import tensorflow as tf
# Done in the tensorflow file via command line